<a href="https://colab.research.google.com/github/Terabyte007/Google_Colab/blob/main/Data_processing_of_Business_Funding_Data_in_Nigeria_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
# ==============================
# Step 1 — Setup & Helpers
# ==============================
import pandas as pd
import numpy as np
import re
import ast

def banner(txt):
    print("\n" + "="*len(txt))
    print(txt)
    print("="*len(txt))

def nrows(df):
    return f"{len(df):,} rows"

# Adjustable FX rates (approximate — update if you wish)
NGN_PER_USD = 1600.0   # 1 USD ≈ 1600 NGN (example)
GBP_PER_USD = 1/1.28   # 1 GBP ≈ 1.28 USD  -> 1 USD ≈ 0.781 GBP
EUR_PER_USD = 1/1.10   # 1 EUR ≈ 1.10 USD  -> 1 USD ≈ 0.909 EUR


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Business Funding Data.csv to Business Funding Data (1).csv


In [ ]:
# ==============================
# Step 2 — Load dataset (with encoding fallback)
# ==============================
banner("Step 2 — Load dataset")

path = "/content/Business Funding Data.csv"  # change if needed

try:
    df = pd.read_csv(path)
except UnicodeDecodeError:
    df = pd.read_csv(path, encoding="latin1")

print("Loaded:", nrows(df))
print("Columns:", list(df.columns))


Step 2 — Load dataset
Loaded: 26 rows
Columns: ['Website Domain', 'Effective date', 'Found At', 'Financing Type', 'Financing Type Normalized', 'Categories', 'Investors', 'Investors Count', 'Amount', 'Amount Normalized', 'Source Urls']


In [ ]:
# ==============================
# Step 3 — Standardize column names
# (lowercase, underscores)
# ==============================
banner("Step 3 — Standardize column names")

df.columns = (
    df.columns.str.strip()
              .str.lower()
              .str.replace(" ", "_")
)

print("Renamed Columns:", list(df.columns))


Step 3 — Standardize column names
Renamed Columns: ['website_domain', 'effective_date', 'found_at', 'financing_type', 'financing_type_normalized', 'categories', 'investors', 'investors_count', 'amount', 'amount_normalized', 'source_urls']


In [ ]:
# ==============================
# Step 4 — Filter for Nigerian businesses
# (.ng domains OR investors mention 'niger' OR source urls mention 'nigeria')
# ==============================
banner("Step 4 — Filter for Nigerian businesses")

orig_rows = len(df)

# make sure expected columns exist; if not, create safe empty ones
for c in ["website_domain", "investors", "source_urls"]:
    if c not in df.columns:
        df[c] = np.nan

mask_ng_domain = df["website_domain"].astype(str).str.contains(r"\.ng\b", case=False, na=False)
mask_investor_ng = df["investors"].astype(str).str.contains(r"niger", case=False, na=False)
mask_source_ng = df["source_urls"].astype(str).str.contains(r"nigeria|\.ng\b", case=False, na=False)

filtered = df[mask_ng_domain | mask_investor_ng | mask_source_ng].copy()

# if filtering removed everything (small dataset), keep all but mark the flag
if filtered.empty:
    print("⚠️ Filter produced 0 rows — keeping all rows but flagging is_nigeria=False.")
    df["is_nigeria"] = False
else:
    df = filtered
    df["is_nigeria"] = True

print("After Nigeria filter:", nrows(df))
print(df[["website_domain", "investors", "source_urls"]].head(5))


Step 4 — Filter for Nigerian businesses
⚠️ Filter produced 0 rows — keeping all rows but flagging is_nigeria=False.
After Nigeria filter: 26 rows
  website_domain                                          investors  \
0  trafigura.com                                                NaN   
1     zenobe.com  avivainvestors.com, lloydsbankinggroup.com, sa...   
2     zenobe.com                                                NaN   
3      canva.com                              stackcapitalgroup.com   
4   fidelity.com                                chevychasetrust.com   

                                         source_urls  
0  https://www.tradefinanceglobal.com/posts/trafi...  
1  https://realassets.ipe.com/news/aviva-among-le...  
2  https://www.innovationnewsnetwork.com/zenobe-a...  
3  https://www.globenewswire.com/news-release/202...  
4  https://www.defenseworld.net/2024/04/11/chevy-...  


In [ ]:
# ==============================
# Step 5 — Handle Missing Values (NO BLANKS LEFT)
# - Replace "N/A"/blanks with NaN
# - Impute investors_count from investors list (if present)
# - Drop rows missing website_domain (if column exists)
# - Fill ALL remaining NaNs with sensible defaults based on dtype/column
# ==============================
banner("Step 5 — Handle Missing Values (No blanks left)")

# 1) Normalize common "blank" tokens to NaN
df.replace(["N/A", "NA", "na", "", " ", "—", "-"], np.nan, inplace=True)

# 2) Impute investors_count from investors list (only if columns exist)
def estimate_investor_count(val):
    if pd.isna(val):
        return np.nan
    s = str(val).strip()
    # Try literal list like "['A', 'B']"
    if s.startswith("[") and s.endswith("]"):
        try:
            import ast
            parsed = ast.literal_eval(s)
            return len(parsed) if isinstance(parsed, list) else np.nan
        except Exception:
            pass
    # Fallback: comma-separated list
    parts = [p.strip() for p in s.split(",") if p.strip()]
    return len(parts) if parts else np.nan

if "investors_count" in df.columns:
    if "investors" in df.columns:
        before_missing_ic = df["investors_count"].isna().sum()
        df["investors_count"] = df["investors_count"].fillna(df["investors"].apply(estimate_investor_count))
        after_missing_ic = df["investors_count"].isna().sum()
        print(f"Imputed investors_count: {before_missing_ic} -> {after_missing_ic} missing")
    # If still NaN, set to 0
    df["investors_count"] = df["investors_count"].fillna(0)
    # Cast to int safely
    df["investors_count"] = pd.to_numeric(df["investors_count"], errors="coerce").fillna(0).astype(int)

# 3) Drop rows missing website_domain (only if the column exists)
if "website_domain" in df.columns:
    before = len(df)
    df = df.dropna(subset=["website_domain"]).copy()
    print(f"Dropped rows missing website_domain: {before - len(df)}")

print("Remaining rows:", nrows(df))

# 4) Fill date-like columns with a placeholder string (to be converted in Step 8)
#    We only touch columns that exist; conversion to datetime happens later.
date_like_cols = [c for c in df.columns if any(k in c for k in ["date", "found_at", "last_updated", "deadline", "start", "end"])]
for c in date_like_cols:
    df[c] = df[c].fillna("1900-01-01")

# 5) Fill list-like text columns with "[]", only if present
for c in ["investors", "categories"]:
    if c in df.columns:
        df[c] = df[c].fillna("[]")

# 6) Now fill all remaining NaNs by dtype, **only on existing columns**
#    - object → "Unknown"
#    - numeric → 0
#    (Date columns were handled above as strings; they’ll be parsed in Step 8)
obj_cols = df.select_dtypes(include=["object"]).columns.tolist()
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Specific object columns that are better as "0" text (e.g., 'amount' as text prior to parsing)
for c in ["amount"]:
    if c in obj_cols:
        df[c] = df[c].fillna("0")
        # remove from generic object fill so we don't overwrite
        obj_cols = [x for x in obj_cols if x != c]

# Generic fills
if obj_cols:
    df[obj_cols] = df[obj_cols].fillna("Unknown")
if num_cols:
    df[num_cols] = df[num_cols].fillna(0)

# 7) Final safety net: if *any* NaNs remain, fill by inferred dtype
if df.isna().any().any():
    for c in df.columns:
        if df[c].isna().any():
            if pd.api.types.is_numeric_dtype(df[c]):
                df[c] = df[c].fillna(0)
            else:
                df[c] = df[c].fillna("Unknown")

# Sanity check
total_nas = int(df.isna().sum().sum())
print(f"✅ NaNs remaining after Step 5: {total_nas}")


Step 5 — Handle Missing Values (No blanks left)
Imputed investors_count: 13 -> 13 missing
Dropped rows missing website_domain: 0
Remaining rows: 26 rows
✅ NaNs remaining after Step 5: 0


In [ ]:
# ==============================
# Step 6 — Normalize Currency Values to USD
# - Clean symbols ($, ₦, £, €)
# - Handle multipliers (k, m, b)
# - Use amount_normalized if already numeric; else parse amount
# ==============================
banner("Step 6 — Normalize Currency Values to USD")

def parse_amount_text_to_usd(text):
    """Parse 'Amount' free text to USD float using simple rules & FX."""
    if pd.isna(text):
        return np.nan
    s = str(text).lower().strip()
    s = s.replace("us$", "$")          # normalize
    s = s.replace("�", "")             # drop corrupted char
    s = s.replace(",", "").replace(" ", "")

    # Detect currency
    cur = "usd"
    if "ngn" in s or "₦" in s:
        cur = "ngn"
    elif "£" in s or "gbp" in s:
        cur = "gbp"
    elif "€" in s or "eur" in s:
        cur = "eur"
    elif "$" in s or "usd" in s:
        cur = "usd"

    # Strip currency symbols/letters except multipliers
    s = s.replace("₦", "").replace("£", "").replace("€", "").replace("$", "")
    s = re.sub(r"(usd|ngn|gbp|eur)", "", s)

    # Multiplier
    mult = 1.0
    if re.search(r"[k]", s): mult = 1e3
    if re.search(r"[m]", s): mult = 1e6
    if re.search(r"[b]", s): mult = 1e9
    s = re.sub(r"[kmb]", "", s)

    try:
        base = float(s) * mult
    except Exception:
        return np.nan

    # Convert to USD if needed
    if cur == "ngn":
        return base / NGN_PER_USD
    elif cur == "gbp":
        return base * (1/GBP_PER_USD)  # GBP -> USD (since GBP_PER_USD is USD->GBP)
    elif cur == "eur":
        return base * (1/ EUR_PER_USD) # EUR -> USD
    else:
        return base  # assume USD

# If 'amount_normalized' is numeric, use it; else parse from 'amount'
df["amount_normalized"] = pd.to_numeric(df["amount_normalized"], errors="coerce")
parsed_amount = df["amount"].apply(parse_amount_text_to_usd)

df["amount_usd"] = np.where(
    df["amount_normalized"].notna(),
    df["amount_normalized"].astype(float),
    parsed_amount
)

# Drop rows where we still couldn't get a number
before_amt = len(df)
df = df.dropna(subset=["amount_usd"]).copy()
after_amt = len(df)

print(f"Rows dropped for missing amount_usd: {before_amt - after_amt}")
print("amount_usd summary:")
print(df["amount_usd"].describe())



Step 6 — Normalize Currency Values to USD
Rows dropped for missing amount_usd: 0
amount_usd summary:
count    2.600000e+01
mean     2.264687e+08
std      5.383239e+08
min      1.600000e+06
25%      4.685750e+06
50%      1.160000e+07
75%      4.750000e+07
max      2.000000e+09
Name: amount_usd, dtype: float64


In [ ]:
# ==============================
# Step 7 — Parse List Fields
# - Convert 'categories' and 'investors' to Python lists
# - Extract investor domains (best-effort)
# ==============================
banner("Step 7 — Parse List Fields")

for c in ["categories", "investors"]:
    if c not in df.columns:
        df[c] = np.nan

def parse_listish(val):
    if pd.isna(val):
        return []
    s = str(val).strip()
    # Try literal list first
    if s.startswith("[") and s.endswith("]"):
        try:
            out = ast.literal_eval(s)
            if isinstance(out, list):
                return [str(x).strip() for x in out]
        except Exception:
            pass
    # Fallback: comma-separated
    parts = [p.strip() for p in s.split(",") if p.strip()]
    return parts

df["categories_parsed"] = df["categories"].apply(parse_listish)
df["investors_parsed"] = df["investors"].apply(parse_listish)

def extract_domain(token):
    token = str(token)
    if "@" in token:
        return token.split("@")[-1].strip().lower()
    # crude URL/domain pick
    m = re.search(r"(https?://)?([^/\s]+)", token)
    return m.group(2).lower() if m else token.lower()

df["investor_domains"] = df["investors_parsed"].apply(lambda lst: [extract_domain(x) for x in lst])

print(df[["categories", "categories_parsed", "investors", "investors_parsed"]].head(5))


Step 7 — Parse List Fields
           categories categories_parsed  \
0                  []                []   
1                  []                []   
2  ["private_equity"]  [private_equity]   
3                  []                []   
4                  []                []   

                                           investors  \
0                                                 []   
1  avivainvestors.com, lloydsbankinggroup.com, sa...   
2                                                 []   
3                              stackcapitalgroup.com   
4                                chevychasetrust.com   

                                    investors_parsed  
0                                                 []  
1  [avivainvestors.com, lloydsbankinggroup.com, s...  
2                                                 []  
3                            [stackcapitalgroup.com]  
4                              [chevychasetrust.com]  


In [ ]:
# Step 8 — Convert Dates & Create Date Features

# List of date columns to try converting
date_cols = ["effective_date", "found_at", "last_updated"]

for c in date_cols:
    if c in df.columns:
        # Force conversion with UTC to avoid the mixed time zone warning
        df[c] = pd.to_datetime(df[c], errors="coerce", utc=True)

# Primary event date = effective_date if available, else found_at
df["event_date"] = df["effective_date"].fillna(df.get("found_at"))

# Ensure event_date is datetime
df["event_date"] = pd.to_datetime(df["event_date"], errors="coerce", utc=True)

# Now safely extract features
df["year"] = df["event_date"].dt.year
df["month"] = df["event_date"].dt.month
df["days_since_funding"] = (pd.Timestamp.utcnow().normalize() - df["event_date"]).dt.days

print("✅ Date conversion successful. Date features created.")

✅ Date conversion successful. Date features created.


In [ ]:
# ==============================
# Step 9 — Remove Duplicates
# - Based on (website_domain, amount_usd)
# - Keep most recent (event_date desc)
# ==============================
banner("Step 9 — Remove Duplicates")

before_dups = len(df)
df = df.sort_values(by=["event_date"], ascending=False)
df = df.drop_duplicates(subset=["website_domain", "amount_usd"], keep="first").copy()
after_dups = len(df)

print(f"Duplicates removed: {before_dups - after_dups}")
print("Remaining:", nrows(df))


Step 9 — Remove Duplicates
Duplicates removed: 0
Remaining: 26 rows


In [ ]:
# ==============================
# Step 10 — Create Derived Features
# - funding_category from financing_type_normalized
# - funding_size_category from amount_usd
# ==============================
banner("Step 10 — Create Derived Features")

if "financing_type_normalized" not in df.columns:
    df["financing_type_normalized"] = np.nan

df["funding_category"] = df["financing_type_normalized"].fillna("Unknown").astype(str).str.title()

def funding_size(x):
    if pd.isna(x):
        return "Unknown"
    if x < 1e5:
        return "Small"
    if x < 1e6:
        return "Medium"
    if x < 1e7:
        return "Large"
    return "Mega"

df["funding_size_category"] = df["amount_usd"].apply(funding_size)

print(df[["amount_usd", "funding_category", "funding_size_category"]].head(5))


Step 10 — Create Derived Features
    amount_usd funding_category funding_size_category
23  13200000.0         Series_A                  Mega
21  10000000.0          Unknown                  Mega
18   1600000.0          Unknown                 Large
5   27200000.0         Series_B                  Mega
19  17400000.0          Unknown                  Mega


In [ ]:
# ==============================
# Step 11 — Save & Quick QA
# ==============================
banner("Step 11 — Save & Quick QA")

out_path = "/content/Business_Funding_Data_Cleaned_afeez.csv"
df.to_csv(out_path, index=False)
print(f"✅ Saved: {out_path}")
print("Final shape:", nrows(df))

# Simple QA summaries
print("\nTop funding categories:")
print(df["funding_category"].value_counts(dropna=False).head(10))

print("\nFunding size buckets:")
print(df["funding_size_category"].value_counts(dropna=False))

print("\nMissing % in key columns:")
for c in ["website_domain", "amount_usd", "event_date", "investors_count"]:
    miss = df[c].isna().mean()*100 if c in df.columns else None
    print(f" - {c}: {miss:.2f}%")


Step 11 — Save & Quick QA
✅ Saved: /content/Business_Funding_Data_Cleaned_afeez.csv
Final shape: 26 rows

Top funding categories:
funding_category
Unknown      18
Seed          4
Series_A      1
Series_B      1
Series_A2     1
Series_I      1
Name: count, dtype: int64

Funding size buckets:
funding_size_category
Mega     15
Large    11
Name: count, dtype: int64

Missing % in key columns:
 - website_domain: 0.00%
 - amount_usd: 0.00%
 - event_date: 0.00%
 - investors_count: 0.00%


In [51]:
# Download the cleaned CSV to your computer
from google.colab import files
files.download("/content/Business_Funding_Data_Cleaned_afeez.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [52]:
df.isna().sum()

,0
website_domain,0
effective_date,6
found_at,0
financing_type,0
financing_type_normalized,0
categories,0
investors,0
investors_count,0
amount,0
amount_normalized,0
